In [35]:
import sys
import os
sys.path.append("../code/")
import cv2
from skimage.metrics import structural_similarity as sk_ssim
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import json
import dataframe_image as dfi
from forwardD_v1 import wm_au_emb
from detectD_v1 import *
from recoveryD_v1 import *
from sklearn.metrics import precision_score,recall_score

import tensorflow as tf

In [20]:
def eval_step(engine, batch):
    return batch

default_evaluator = Engine(eval_step)

In [9]:
root_dir = "D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images"
list_paths = []
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        list_paths.append(os.path.join(subdir, file))

In [10]:
print(list_paths)

['D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\airplane.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\baboon.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\house.tiff', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\lena.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\peppers.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\sailboat.tiff', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\splash.tiff', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\tiffany.bmp']


In [11]:
list_img = []
for i in range(len(list_paths)):
    img = cv2.imread(list_paths[i])
    list_img.append(img)

In [12]:
key = (13,137,811)
list_w_img=[]
for i in range(len(list_paths)):
    img = cv2.imread(list_paths[i])
    w_img = wm_au_emb(img,key=key)
    list_w_img.append(w_img)

In [52]:
A = np.array(list_img)

In [54]:
print(A.shape)

(8, 512, 512, 3)


In [64]:
X_host = torch.tensor(np.array(list_img)).permute(0,3,1,2)
Y_water = torch.tensor(np.array(list_w_img)).permute(0,3,1,2)

In [65]:
w_SSIM_tensor = tf.image.ssim(list_img[0],list_w_img[0],max_val=255,
    filter_size=11,
    filter_sigma=1.5,
    k1=0.01,
    k2=0.03,
    return_index_map=False)

In [60]:
print(w_SSIM_torch)

tensor([1., 1., 1., 1., 1., 1., 1., 1.])


In [61]:
w_PSNR = []
w_SSIM = []
for i in range(len(list_img)):
    w_PSNR.append(cv2.PSNR(list_img[i],list_w_img[i]))
    w_SSIM.append(sk_ssim(cv2.cvtColor(list_img[i],cv2.COLOR_BGR2GRAY),cv2.cvtColor(list_w_img[i],cv2.COLOR_BGR2GRAY),data_range=255.0))


In [63]:
print(np.mean(w_PSNR))
print(w_SSIM)
print(w_SSIM_torch)

46.32122766977977
[0.9925447019165354, 0.9980574723101749, 0.9946822017850739, 0.9937791771271932, 0.993973957033285, 0.9954416517240117, 0.9919054097888359, 0.9929904380491563]
tensor([1., 1., 1., 1., 1., 1., 1., 1.])


In [ ]:
# df_all_PSNR.to_csv("results/PSNR_Watermarked_Images.csv")
# df_all_ssim.to_csv("results/SSIM_Watermarked_Images.csv")

In [69]:
N = 512
np.random.seed(1)
t_all_image = np.random.randint(
    low=0, 
    high=256,
    size=(N, N, 3),
    dtype=np.uint8
)

In [67]:
list_tr = [10,20,30,40,50,60,70,80]
list_d = []
for i in range(len(list_tr)):
    list_d.append(int(np.sqrt(512*512*list_tr[i]/100)/2+ 0.5))
print(list_d)

[81, 114, 140, 162, 181, 198, 214, 229]


In [72]:
list_PSNR_Recovered_all = []
list_SSIM_Recovered_all = []
list_PSNR_Recovered_all_w_original = []
list_SSIM_Recovered_all_w_original = []
list_r3_img_all = []
for itr in range(len(list_d)):
    list_PSNR = []
    list_SSIM = []
    list_PSNR_original = []
    list_SSIM_original = []
    list_r3_img = []
    for index_img in range(len(list_w_img)):
        t_img = list_w_img[index_img].copy()
        d = list_d[itr]
        t_img[256-d:256+d,256-d:256+d,:] =  t_all_image[256-d:256+d,256-d:256+d,:]
        lv1_matrix = level_one_detection(t_img)
        lv2_matrix = lv1_matrix.copy()
        lv3_matrix = lv1_matrix.copy()
        for channel in range(3):
            lv2_matrix[:,:,channel] = level_two_detection(lv1_matrix[:,:,channel])
            lv3_matrix[:,:,channel] = level_three_detection(lv2_matrix[:,:,channel])
        r1_img,lv4_BGR= stage_1_recovery(t_img=t_img,key=(13,137,811))
        mask1 = np.zeros((512,512),dtype = np.uint8)
        for i in range(512):
            for j in range(512):
                if not lv4_BGR[int(i/1),int(j/1),0]:
                    mask1[i,j] = 255
        mask2 = np.zeros((512,512),dtype = np.uint8)
        for i in range(512):
            for j in range(512):
                if not lv4_BGR[int(i/1),int(j/1),1]:
                    mask2[i,j] = 255
        mask3 = np.zeros((512,512),dtype = np.uint8)
        for i in range(512):
            for j in range(512):
                if not lv4_BGR[int(i/1),int(j/1),2]:
                    mask3[i,j] = 255
        r3_img = r1_img.copy()
        r3_img[:,:,0] = cv2.inpaint(r1_img[:,:,0],mask1,0,cv2.INPAINT_TELEA)
        r3_img[:,:,1] = cv2.inpaint(r1_img[:,:,1],mask2,0,cv2.INPAINT_TELEA)
        r3_img[:,:,2] = cv2.inpaint(r1_img[:,:,2],mask3,0,cv2.INPAINT_TELEA)
        list_r3_img.append(r3_img)
        list_PSNR.append(cv2.PSNR(r3_img,list_w_img[index_img]))
        list_SSIM.append(sk_ssim(cv2.cvtColor(r3_img,cv2.COLOR_BGR2GRAY),cv2.cvtColor(list_w_img[index_img],cv2.COLOR_BGR2GRAY)))
        list_PSNR_original.append(cv2.PSNR(r3_img,list_img[index_img]))
        list_SSIM_original.append(sk_ssim(cv2.cvtColor(r3_img,cv2.COLOR_BGR2GRAY),cv2.cvtColor(list_img[index_img],cv2.COLOR_BGR2GRAY)))
    list_PSNR_Recovered_all.append(list_PSNR)
    list_SSIM_Recovered_all.append(list_SSIM)
    list_PSNR_Recovered_all_w_original.append(list_PSNR_original)
    list_SSIM_Recovered_all_w_original.append(list_SSIM_original)
    list_r3_img_all.append(list_r3_img)

In [73]:
list_ssim_recover_all_torch = []
for i in range(len(list_d)):
    Z_recover = torch.tensor(np.array(list_r3_img_all[i])).permute(0,3,1,2)
    list_ssim_recover_all_torch.append(ssim(Z_recover,Y_water,size_average=False))

In [74]:
for i in range(len(list_d)):
    print(list_ssim_recover_all_torch[i])

tensor([1., 1., 1., 1., 1., 1., 1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1.])


In [89]:
list_image_name =  ['Airplane', 'Baboon', 'House', 'Lena','Peppers','Sailboat','Splash','Tiffany','Average']

In [85]:
np.mean(list_PSNR_Recovered_all[3])

29.547761005363554

In [77]:
np.mean(list_PSNR_Recovered_all_w_original[0])

37.99163958245791

In [86]:
print(np.mean(list_PSNR_Recovered_all[2]))

32.51473631435336


In [87]:
dict_PSNR = {}
dict_SSIM = {}

In [90]:
for i in range(8):
    dict_PSNR[list_image_name[i]] = [list_PSNR_Recovered_all[j][i] for j in range(8)]
dict_PSNR[list_image_name[8]] = [np.mean(list_PSNR_Recovered_all[j]) for j in range(8)]

In [91]:
for i in range(8):
    dict_SSIM[list_image_name[i]] = [list_SSIM_Recovered_all[j][i] for j in range(8)]
dict_SSIM[list_image_name[8]] = [np.mean(list_SSIM_Recovered_all[j]) for j in range(8)]

In [92]:
print(dict_PSNR['Average'])

[38.615928981898506, 35.36111792783909, 32.51473631435336, 29.547761005363554, 27.093447420365887, 25.19601474777726, 23.43070582008706, 21.39353825656846]


In [93]:
df_PSNR_all = pd.DataFrame.from_dict(dict_PSNR)

In [94]:
df_SSIM_all = pd.DataFrame.from_dict(dict_SSIM)

In [95]:
df_SSIM_all.to_csv("SSIM_all_recovered1.csv")

In [96]:
df_PSNR_all.to_csv("PSNR_all_recovered1.csv")

In [97]:
def highlight_max(s):
    # Get 5 largest values of the column
    is_large1 = s.nlargest(1).values
    is_large2 = s.nlargest(2).values
    # Apply style is the current value is among the 5 biggest values
    return ['color: DarkRed;font-weight:900' if v in is_large1 else 'font-weight:900' if v in is_large2 else '' for v in s]

In [ ]:
# hl_df_ssim = df_all_ssim.style.apply(highlight_max,subset=df_all_ssim.columns[1:],axis = 1).apply(lambda x: ['background: lightgreen' if x.name == 8 else '' for i in x], 
#                axis=1)

In [98]:
hl_df_PSNR_all = df_PSNR_all.style.apply(highlight_max,subset=df_PSNR_all.columns[:],axis=1)
hl_df_PSNR_all.format(precision=2)
hl_df_PSNR_all.hide_index()

C:\Users\DuyDat\AppData\Local\Temp\ipykernel_16996\3007742081.py:3: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  hl_df_PSNR_all.hide_index()


Airplane,Baboon,House,Lena,Peppers,Sailboat,Splash,Tiffany,Average
37.33,37.69,36.33,38.39,38.27,37.03,43.04,40.84,38.62
34.28,33.33,33.32,34.88,35.82,33.74,39.67,37.85,35.36
31.75,28.87,30.97,32.22,33.62,30.83,36.19,35.67,32.51
28.89,25.13,27.98,29.58,31.01,27.69,32.62,33.49,29.55
26.43,22.62,25.53,27.56,28.53,25.31,29.70,31.06,27.09
24.65,20.83,23.75,25.81,26.54,23.38,27.70,28.91,25.20
23.07,19.42,22.10,24.19,24.12,21.20,26.22,27.13,23.43
21.09,17.98,20.06,22.29,21.90,18.99,23.51,25.32,21.39


In [99]:
hl_df_SSIM_all = df_SSIM_all.style.apply(highlight_max,subset=df_SSIM_all.columns[:],axis=1)
hl_df_SSIM_all.format(precision=4)
hl_df_SSIM_all.hide_index()

C:\Users\DuyDat\AppData\Local\Temp\ipykernel_16996\3815801646.py:3: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  hl_df_SSIM_all.hide_index()


Airplane,Baboon,House,Lena,Peppers,Sailboat,Splash,Tiffany,Average
0.9899,0.9812,0.9868,0.9887,0.9844,0.9838,0.9918,0.9875,0.9868
0.9801,0.9593,0.9708,0.9738,0.9685,0.9694,0.9826,0.9745,0.9724
0.9663,0.9249,0.9520,0.9547,0.9489,0.9490,0.9699,0.9597,0.9532
0.9415,0.8649,0.9188,0.9244,0.9200,0.9160,0.9491,0.9357,0.9213
0.9090,0.7910,0.8793,0.8870,0.8846,0.8714,0.9227,0.9012,0.8808
0.8700,0.7029,0.8279,0.8432,0.8455,0.8148,0.8896,0.8580,0.8315
0.8201,0.5989,0.7642,0.7898,0.7900,0.7404,0.8525,0.8035,0.7699
0.7449,0.4634,0.6692,0.7119,0.7206,0.6334,0.7847,0.7342,0.6828


In [ ]:
# hl_df_ssim.format(precision=4)
# hl_df_ssim.hide_index()

In [102]:
dfi.export(
    hl_df_PSNR_all,
    "results/PSNR_recovered_Images.png",
    fontsize=12,
)

In [103]:
dfi.export(
    hl_df_SSIM_all,
    "results/SSIM_recovered_Images.png",
    fontsize=12,
)